In [2]:
import sys
import pandas as pd
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
# def getWrapper():
#     user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
#     endpoint_url = "https://query.wikidata.org/sparql"
#     sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
#     return sparql
# endpoint_url = "https://query.wikidata.org/sparql"
# sparql = SPARQLWrapper(endpoint_url)
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0")
print("connected")

In [1]:
def getresult(query, count):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0")
#     print("connected")
    sparql.addExtraURITag("timeout","10000")
    sparql.setQuery(query)
#     print("A")
    sparql.setReturnFormat(JSON)
#     print("B")
#     try:
    results = sparql.queryAndConvert()
#     print("C")
#     except Exception as e:
#         print(e)
    results = results['results']['bindings']
    if results:
        new_results = []
        for result in results:
            new_result = {}
            for key in result:
                new_result[key] = result[key]['value']
            new_results.append(new_result)
        return new_results
    else:
        count += 1
#         print("no results")

In [3]:
# Harry Potter and the Philosopher's Stone
query = """
SELECT ?book ?bookLabel ?publicationYLabel ?author ?authorLabel ?birthYear
WHERE {
  ?book rdfs:label "Harry Potter and the Philosopher's Stone"@en.
  ?book wdt:P31 ?worktype; 
        wdt:P50 ?author.
  ?author wdt:P569 ?birthDate.
  BIND(YEAR(?birthDate) AS ?birthYear)
  OPTIONAL { ?book wdt:P577 ?publicationdate.}
  FILTER(?worktype = wd:Q7725634 || ?worktype = wd:Q1279564) #41298 magazine 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
p = getresult(query,0)
print(p)
result_df = pd.DataFrame.from_dict(p)
print(type(result_df))
display(result_df)

[{'book': 'http://www.wikidata.org/entity/Q43361', 'author': 'http://www.wikidata.org/entity/Q34660', 'birthYear': '1965', 'bookLabel': "Harry Potter and the Philosopher's Stone", 'authorLabel': 'J. K. Rowling'}]
<class 'pandas.core.frame.DataFrame'>


,book,author,birthYear,bookLabel,authorLabel
0,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q34660,1965,Harry Potter and the Philosopher's Stone,J. K. Rowling


In [2]:
metadata = pd.read_csv("metadata.csv")
metadata = metadata.rename(columns={"Unnamed: 1" : "found"})
# display(metadata)

/tmp/ipykernel_134/3695526248.py:1: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("metadata.csv")


In [17]:
test = metadata.iloc[71394:]
base_query = """
SELECT ?book ?bookLabel ?author ?authorLabel ?birthYear
WHERE {{
  ?book rdfs:label "{}"@en.
  ?book wdt:P31 ?worktype; 
        wdt:P50 ?author.
  ?author wdt:P569 ?birthDate.
  BIND(YEAR(?birthDate) AS ?birthYear)
  FILTER(?worktype = wd:Q7725634 || ?worktype = wd:Q1279564 || ?worktype = wd:Q47461344 || ?worktype = wd:Q27217074 || ?worktype = wd:Q732577|| ?worktype = wd:Q8261) #41298 magazine 
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
}}
"""
count = 0
for index, row in test.iterrows():
    book_title = row['title']
    if pd.notna(book_title):
        book_title = book_title.split(':')[0]
        book_title = book_title.split(';')[0]
        book_title = book_title.split(',')[0]
        book_title = book_title.split('"')[0]
        print(book_title)
        if (book_title != "Punchinello") and (book_title != "") and pd.notna(book_title):
            formatted_query = base_query.format(book_title)
            df_1 = getresult(formatted_query, count)
            if df_1 is not None:
                rdf = pd.DataFrame.from_dict(df_1)
                result_df = pd.concat([result_df,rdf])
print(count)
display(result_df)

Charlotte's Inheritance
United States Presidents' Inaugural Speeches
Prometheus
Les grandes dames
Le corricolo
In the Midst of Alarms
Vand- og stenhoejsplanter en vejledning for havevenner
The Atlantic Monthly
Scientific American Supplement
Chip
Omoo
Typee
Ten Thousand Dreams Interpreted
The Battle of the Books
The Bickerstaff-Partridge Papers
Gulliver's Travels
A Modest Proposal
The Journal to Stella
A Tale of a Tub
Three Sermons
Crucial Instances
Bunner Sisters
Fighting France
The Lamplighter
The House of Mirth
The Hermit and the Wild Woman
Madame de Treymes
Artemis to Actaeon and Other Verses
Sanctuary
Summer
The Descent of Man and Other Stories
The Glimpses of the Moon
Ethan Frome
Tales of Men and Ghosts
Alice's Adventures in Wonderland
The Touchstone
The Reef
The Valley of Decision
Early Short Fiction of Edith Wharton
The Exploring Expedition to the Rocky Mountains
A Night Out
Clarissa Harlowe
The Orange-Yellow Diamond
Life and Death of Harriett Frean
Italian Letters
The Real Cybe

Personal Poems I
Personal Poems II
Occasional Poems
The Tent on the Beach
At Sundown
Personal Poems
Margaret Smith's Journal
My Summer with Dr. Singletary
Tales and Sketches
Rinkitink in Oz
Margaret Smith's Journal
Old Portraits and Modern Sketches
Personal Sketches and Tributes
Historical Papers
The Works of John Greenleaf Whittier
The Conflict with Slavery
Reform and Politics
The Inner Life
Criticism
The Works of John Greenleaf Whittier
The Lost Princess of Oz
The Prisoner of Zenda
The Works of John Greenleaf Whittier
The Poetical Works of Alexander Pope
History of the French Revolution from 1789 to 1814
Hung Lou Meng
Hung Lou Meng
Chico
A Little Book of Western Verse
From Boyhood to Manhood
The Cords of Vanity
Joseph Andrews
The Tin Woodman of Oz
The Elegies of Tibullus
Joseph Andrews
The Botanic Garden
The Young Buglers
The Case of Richard Meynell
The Diary of a Superfluous Man and Other Stories
Ramuntcho
Echoes of the War
The Field of Ice
Who Can Be Happy and Free in Russia?
Glind

The Mirror of Literature
The Mirror of Literature
A Theological-Political Treatise [Part III]
The Garden of Bright Waters
The Works of Lord Byron
The Mirror of Literature
The Box with Broken Seals
Viviette
Black Jack
The Two Guardians
The Bronze Bell
A Chair on the Boulevard
The Great Events by Famous Historians
A Theological-Political Treatise [Part IV]
Groups of the Order p^m Which Contain Cyclic Subgroups of Order p^(m-3)
K
The Last Trail
A Treatise on the Theory of Invariants
A Primer of Quaternions
Woman's Institute Library of Cookery. Volume 1
Woman's Institute Library of Cookery. Volume 2
Woman's Institute Library of Cookery. Volume 3
Woman's Institute Library of Cookery. Volume 4
Woman's Institute Library of Cookery. Volume 5
Malbone
Life in London
Biography of a Slave
Lectures on Ten British Mathematicians of the Nineteenth Century
Journal of an Expedition into the Interior of Tropical Australia
The Conquest of Fear
Histoire de la Révolution française
The Atlantic Monthly
Quee

,book,author,birthYear,bookLabel,authorLabel,authorLast,book_QN,author_QN
0,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q34660,1965,Harry Potter and the Philosopher's Stone,J. K. Rowling,Rowling,Q43361,Q34660
0,http://www.wikidata.org/entity/Q7207360,http://www.wikidata.org/entity/Q178698,1907,Poems,W. H. Auden,Auden,Q7207360,Q178698
1,http://www.wikidata.org/entity/Q7207361,http://www.wikidata.org/entity/Q25973,1877,Poems,Hermann Hesse,Hesse,Q7207361,Q25973
2,http://www.wikidata.org/entity/Q7207362,http://www.wikidata.org/entity/Q178106,1883,Poems,William Carlos Williams,Williams,Q7207362,Q178106
3,http://www.wikidata.org/entity/Q7207363,http://www.wikidata.org/entity/Q44183,1911,Poems,William Golding,Golding,Q7207363,Q44183
...,...,...,...,...,...,...,...,...
25,http://www.wikidata.org/entity/Q117450610,http://www.wikidata.org/entity/Q45546,1770,Poems,William Wordsworth,NaN,NaN,NaN
26,http://www.wikidata.org/entity/Q117458723,http://www.wikidata.org/entity/Q2978887,1779,Poems,Clement Clarke Moore,NaN,NaN,NaN
27,http://www.wikidata.org/entity/Q117486968,http://www.wikidata.org/entity/Q5648827,1788,Poems,Hannah Flagg Gould,NaN,NaN,NaN
28,http://www.wikidata.org/entity/Q117748698,http://www.wikidata.org/entity/Q3664258,1835,Poems,Celia Thaxter,NaN,NaN,NaN


In [18]:
result_df['authorLast'] = result_df['authorLabel'].str.split(' ').str[-1]
result_df['book_QN'] = result_df['book'].str.split('/').str[-1]
result_df['author_QN'] = result_df['author'].str.split('/').str[-1]
display(result_df)

,book,author,birthYear,bookLabel,authorLabel,authorLast,book_QN,author_QN
0,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q34660,1965,Harry Potter and the Philosopher's Stone,J. K. Rowling,Rowling,Q43361,Q34660
0,http://www.wikidata.org/entity/Q7207360,http://www.wikidata.org/entity/Q178698,1907,Poems,W. H. Auden,Auden,Q7207360,Q178698
1,http://www.wikidata.org/entity/Q7207361,http://www.wikidata.org/entity/Q25973,1877,Poems,Hermann Hesse,Hesse,Q7207361,Q25973
2,http://www.wikidata.org/entity/Q7207362,http://www.wikidata.org/entity/Q178106,1883,Poems,William Carlos Williams,Williams,Q7207362,Q178106
3,http://www.wikidata.org/entity/Q7207363,http://www.wikidata.org/entity/Q44183,1911,Poems,William Golding,Golding,Q7207363,Q44183
...,...,...,...,...,...,...,...,...
25,http://www.wikidata.org/entity/Q117450610,http://www.wikidata.org/entity/Q45546,1770,Poems,William Wordsworth,Wordsworth,Q117450610,Q45546
26,http://www.wikidata.org/entity/Q117458723,http://www.wikidata.org/entity/Q2978887,1779,Poems,Clement Clarke Moore,Moore,Q117458723,Q2978887
27,http://www.wikidata.org/entity/Q117486968,http://www.wikidata.org/entity/Q5648827,1788,Poems,Hannah Flagg Gould,Gould,Q117486968,Q5648827
28,http://www.wikidata.org/entity/Q117748698,http://www.wikidata.org/entity/Q3664258,1835,Poems,Celia Thaxter,Thaxter,Q117748698,Q3664258


In [19]:
result_df.to_csv('From71394.csv', index=False)

Read in the file from start to 31290 saved

In [24]:
first = pd.read_csv('From71394.csv')
display(first)

,book,author,birthYear,bookLabel,authorLabel,authorLast,book_QN,author_QN
0,http://www.wikidata.org/entity/Q43361,http://www.wikidata.org/entity/Q34660,1965.0,Harry Potter and the Philosopher's Stone,J. K. Rowling,Rowling,Q43361,Q34660
1,http://www.wikidata.org/entity/Q7207360,http://www.wikidata.org/entity/Q178698,1907.0,Poems,W. H. Auden,Auden,Q7207360,Q178698
2,http://www.wikidata.org/entity/Q7207361,http://www.wikidata.org/entity/Q25973,1877.0,Poems,Hermann Hesse,Hesse,Q7207361,Q25973
3,http://www.wikidata.org/entity/Q7207362,http://www.wikidata.org/entity/Q178106,1883.0,Poems,William Carlos Williams,Williams,Q7207362,Q178106
4,http://www.wikidata.org/entity/Q7207363,http://www.wikidata.org/entity/Q44183,1911.0,Poems,William Golding,Golding,Q7207363,Q44183
...,...,...,...,...,...,...,...,...
3641,http://www.wikidata.org/entity/Q117450610,http://www.wikidata.org/entity/Q45546,1770.0,Poems,William Wordsworth,Wordsworth,Q117450610,Q45546
3642,http://www.wikidata.org/entity/Q117458723,http://www.wikidata.org/entity/Q2978887,1779.0,Poems,Clement Clarke Moore,Moore,Q117458723,Q2978887
3643,http://www.wikidata.org/entity/Q117486968,http://www.wikidata.org/entity/Q5648827,1788.0,Poems,Hannah Flagg Gould,Gould,Q117486968,Q5648827
3644,http://www.wikidata.org/entity/Q117748698,http://www.wikidata.org/entity/Q3664258,1835.0,Poems,Celia Thaxter,Thaxter,Q117748698,Q3664258


In [25]:
metadata['bookLabel'] = metadata['title'].str.split(':').str[0]
metadata['bookLabel'] = metadata['bookLabel'].str.split(';').str[0]
metadata['bookLabel'] = metadata['bookLabel'].str.split(',').str[0]
metadata['bookLabel'] = metadata['bookLabel'].str.split('"').str[0]
metadata['authorLast'] = metadata['author'].str.split(',').str[0]
data1 = metadata.iloc[71394:]
display(data1)

,id,found,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,bookLabel,authorLast
71394,PG9259,NaN,Charlotte's Inheritance,"Braddon, M. E. (Mary Elizabeth)",1835.0,1915.0,['en'],32,{'Fiction'},NaN,Charlotte's Inheritance,Braddon
71395,PG925,NaN,United States Presidents' Inaugural Speeches: ...,United States. Presidents,NaN,NaN,['en'],373,{'Presidents -- United States -- Inaugural add...,NaN,United States Presidents' Inaugural Speeches,United States. Presidents
71396,PG9260,NaN,Prometheus: Dramatisches Fragment,"Goethe, Johann Wolfgang von",1749.0,1832.0,['de'],78,{'Drama'},NaN,Prometheus,Goethe
71397,PG9261,NaN,Les grandes dames,"Houssaye, Arsène",1815.0,1896.0,['fr'],15,"{'Nobility -- France -- Fiction', 'Good and ev...",NaN,Les grandes dames,Houssaye
71398,PG9262,NaN,Le corricolo,"Dumas, Alexandre",1802.0,1870.0,['fr'],56,{'Naples (Italy) -- Description and travel'},NaN,Le corricolo,Dumas
...,...,...,...,...,...,...,...,...,...,...,...,...
72214,PG9998,NaN,Poems,"Betham, Matilda",1776.0,1852.0,['en'],22,{'Poetry'},NaN,Poems,Betham
72215,PG9999,NaN,"Harriet, the Moses of Her People","Bradford, Sarah H. (Sarah Hopkins)",1818.0,1912.0,['en'],88,"{'Tubman, Harriet, 1822-1913', 'Enslaved perso...",NaN,Harriet,Bradford
72216,PG999,NaN,Divina Commedia di Dante: Paradiso,Dante Alighieri,1265.0,1321.0,['it'],57,"{'Italian poetry -- To 1400', 'Epic poetry, It...",NaN,Divina Commedia di Dante,Dante Alighieri
72217,PG99,NaN,Collected Articles of Frederick Douglass,"Douglass, Frederick",1818.0,1895.0,['en'],206,"{'Essays', 'Reconstruction (U.S. history, 1865...",NaN,Collected Articles of Frederick Douglass,Douglass


In [26]:
result1 = pd.merge(data1,first, on = ['bookLabel','authorLast'])
display(result1)

,id,found,title,author_x,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,bookLabel,authorLast,book,author_y,birthYear,authorLabel,book_QN,author_QN
0,PG9260,NaN,Prometheus: Dramatisches Fragment,"Goethe, Johann Wolfgang von",1749.0,1832.0,['de'],78,{'Drama'},NaN,Prometheus,Goethe,http://www.wikidata.org/entity/Q2112787,http://www.wikidata.org/entity/Q5879,1749.0,Johann Wolfgang von Goethe,Q2112787,Q5879
1,PG9260,NaN,Prometheus: Dramatisches Fragment,"Goethe, Johann Wolfgang von",1749.0,1832.0,['de'],78,{'Drama'},NaN,Prometheus,Goethe,http://www.wikidata.org/entity/Q108394484,http://www.wikidata.org/entity/Q5879,1749.0,Johann Wolfgang von Goethe,Q108394484,Q5879
2,PG9263,NaN,In the Midst of Alarms,"Barr, Robert",1850.0,1912.0,['en'],15,{'Canada -- Fiction'},NaN,In the Midst of Alarms,Barr,http://www.wikidata.org/entity/Q124083904,http://www.wikidata.org/entity/Q650048,1849.0,Robert Barr,Q124083904,Q650048
3,PG9263,NaN,In the Midst of Alarms,"Barr, Robert",1850.0,1912.0,['en'],15,{'Canada -- Fiction'},NaN,In the Midst of Alarms,Barr,http://www.wikidata.org/entity/Q124083904,http://www.wikidata.org/entity/Q650048,1850.0,Robert Barr,Q124083904,Q650048
4,PG9268,NaN,Omoo: Adventures in the South Seas,"Melville, Herman",1819.0,1891.0,['en'],12,"{'Adventure stories', 'Polynesia -- Descriptio...",NaN,Omoo,Melville,http://www.wikidata.org/entity/Q3882301,http://www.wikidata.org/entity/Q4985,1819.0,Herman Melville,Q3882301,Q4985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,PG9976,NaN,Hernani,"Hugo, Victor",1802.0,1885.0,['fr'],121,"{'Hernani (Fictitious character) -- Drama', 'T...",NaN,Hernani,Hugo,http://www.wikidata.org/entity/Q1125255,http://www.wikidata.org/entity/Q535,1802.0,Victor Hugo,Q1125255,Q535
281,PG9990,NaN,"Brave and Bold; Or, The Fortunes of Robert Rus...","Alger, Horatio, Jr.",1832.0,1899.0,['en'],26,"{'Missing persons -- Juvenile fiction', 'Adven...",NaN,Brave and Bold,Alger,http://www.wikidata.org/entity/Q12857847,http://www.wikidata.org/entity/Q453251,1832.0,Horatio Alger,Q12857847,Q453251
282,PG9997,NaN,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],25,{'Canada -- History -- To 1763 (New France)'},NaN,France and England in North America,Parkman,http://www.wikidata.org/entity/Q5478317,http://www.wikidata.org/entity/Q1340161,1823.0,Francis Parkman,Q5478317,Q1340161
283,PG9997,NaN,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],25,{'Canada -- History -- To 1763 (New France)'},NaN,France and England in North America,Parkman,http://www.wikidata.org/entity/Q5478317,http://www.wikidata.org/entity/Q1340161,1823.0,Francis Parkman,Q5478317,Q1340161


In [27]:
result1.to_csv('resultEnd.csv', index=False)

Save the matching results

In [ ]:
book_author = {}
book = {}
for index, row in metadata.iterrows():
    book_author[(row['title'],row['author'])] = ""
    book[row['title']] = ""

In [ ]:
for index, row in df.iterrows():
    book[row['bookLabel']] = row['book']
    print(row['bookLabel'],book[row['bookLabel']])

In [3]:
authorLastMatch = pd.read_csv('resultEnd.csv')
display(authorLastMatch)

,id,found,title,author_x,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,bookLabel,authorLast,book,author_y,birthYear,authorLabel,book_QN,author_QN
0,PG9260,NaN,Prometheus: Dramatisches Fragment,"Goethe, Johann Wolfgang von",1749.0,1832.0,['de'],78,{'Drama'},NaN,Prometheus,Goethe,http://www.wikidata.org/entity/Q2112787,http://www.wikidata.org/entity/Q5879,1749.0,Johann Wolfgang von Goethe,Q2112787,Q5879
1,PG9260,NaN,Prometheus: Dramatisches Fragment,"Goethe, Johann Wolfgang von",1749.0,1832.0,['de'],78,{'Drama'},NaN,Prometheus,Goethe,http://www.wikidata.org/entity/Q108394484,http://www.wikidata.org/entity/Q5879,1749.0,Johann Wolfgang von Goethe,Q108394484,Q5879
2,PG9263,NaN,In the Midst of Alarms,"Barr, Robert",1850.0,1912.0,['en'],15,{'Canada -- Fiction'},NaN,In the Midst of Alarms,Barr,http://www.wikidata.org/entity/Q124083904,http://www.wikidata.org/entity/Q650048,1849.0,Robert Barr,Q124083904,Q650048
3,PG9263,NaN,In the Midst of Alarms,"Barr, Robert",1850.0,1912.0,['en'],15,{'Canada -- Fiction'},NaN,In the Midst of Alarms,Barr,http://www.wikidata.org/entity/Q124083904,http://www.wikidata.org/entity/Q650048,1850.0,Robert Barr,Q124083904,Q650048
4,PG9268,NaN,Omoo: Adventures in the South Seas,"Melville, Herman",1819.0,1891.0,['en'],12,"{'Adventure stories', 'Polynesia -- Descriptio...",NaN,Omoo,Melville,http://www.wikidata.org/entity/Q3882301,http://www.wikidata.org/entity/Q4985,1819.0,Herman Melville,Q3882301,Q4985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,PG9976,NaN,Hernani,"Hugo, Victor",1802.0,1885.0,['fr'],121,"{'Hernani (Fictitious character) -- Drama', 'T...",NaN,Hernani,Hugo,http://www.wikidata.org/entity/Q1125255,http://www.wikidata.org/entity/Q535,1802.0,Victor Hugo,Q1125255,Q535
281,PG9990,NaN,"Brave and Bold; Or, The Fortunes of Robert Rus...","Alger, Horatio, Jr.",1832.0,1899.0,['en'],26,"{'Missing persons -- Juvenile fiction', 'Adven...",NaN,Brave and Bold,Alger,http://www.wikidata.org/entity/Q12857847,http://www.wikidata.org/entity/Q453251,1832.0,Horatio Alger,Q12857847,Q453251
282,PG9997,NaN,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],25,{'Canada -- History -- To 1763 (New France)'},NaN,France and England in North America,Parkman,http://www.wikidata.org/entity/Q5478317,http://www.wikidata.org/entity/Q1340161,1823.0,Francis Parkman,Q5478317,Q1340161
283,PG9997,NaN,"France and England in North America, Part III:...","Parkman, Francis",1823.0,1893.0,['en'],25,{'Canada -- History -- To 1763 (New France)'},NaN,France and England in North America,Parkman,http://www.wikidata.org/entity/Q5478317,http://www.wikidata.org/entity/Q1340161,1823.0,Francis Parkman,Q5478317,Q1340161


In [4]:
display(authorLastMatch[authorLastMatch['authoryearofbirth'] != authorLastMatch['birthYear']])

,id,found,title,author_x,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,bookLabel,authorLast,book,author_y,birthYear,authorLabel,book_QN,author_QN
2,PG9263,NaN,In the Midst of Alarms,"Barr, Robert",1850.0,1912.0,['en'],15,{'Canada -- Fiction'},NaN,In the Midst of Alarms,Barr,http://www.wikidata.org/entity/Q124083904,http://www.wikidata.org/entity/Q650048,1849.0,Robert Barr,Q124083904,Q650048
28,PG9298,NaN,Life and Death of Harriett Frean,"Sinclair, May",1863.0,1946.0,['en'],49,"{'Repression (Psychology) -- Fiction', 'Single...",NaN,Life and Death of Harriett Frean,Sinclair,http://www.wikidata.org/entity/Q19572638,http://www.wikidata.org/entity/Q1131497,1870.0,May Sinclair,Q19572638,Q1131497
32,PG9312,NaN,From Whose Bourne,"Barr, Robert",1850.0,1912.0,['en'],25,"{'Ghosts -- Fiction', 'Mystery fiction'}",NaN,From Whose Bourne,Barr,http://www.wikidata.org/entity/Q124083256,http://www.wikidata.org/entity/Q650048,1849.0,Robert Barr,Q124083256,Q650048
115,PG9489,NaN,Michael O'Halloran,"Stratton-Porter, Gene",1863.0,1924.0,['en'],16,"{'City and town life -- Fiction', 'Didactic fi...",NaN,Michael O'Halloran,Stratton-Porter,http://www.wikidata.org/entity/Q23307585,http://www.wikidata.org/entity/Q5531480,1868.0,Gene Stratton-Porter,Q23307585,Q5531480
180,PG9678,NaN,The Consolidator,"Defoe, Daniel",NaN,1731.0,['en'],10,"{'Satire', 'Fantasy fiction', 'Interplanetary ...",NaN,The Consolidator,Defoe,http://www.wikidata.org/entity/Q60753482,http://www.wikidata.org/entity/Q40946,1660.0,Daniel Defoe,Q60753482,Q40946
181,PG9678,NaN,The Consolidator,"Defoe, Daniel",NaN,1731.0,['en'],10,"{'Satire', 'Fantasy fiction', 'Interplanetary ...",NaN,The Consolidator,Defoe,http://www.wikidata.org/entity/Q60753482,http://www.wikidata.org/entity/Q40946,1660.0,Daniel Defoe,Q60753482,Q40946
186,PG9683,NaN,A Journal of the Plague Year,"Defoe, Daniel",NaN,1731.0,['en'],20,"{'Plague -- Fiction', 'Historical fiction', 'G...",NaN,A Journal of the Plague Year,Defoe,http://www.wikidata.org/entity/Q1215399,http://www.wikidata.org/entity/Q40946,1660.0,Daniel Defoe,Q1215399,Q40946
187,PG9685,NaN,Moll Flanders,"Defoe, Daniel",NaN,1731.0,['en'],21,"{'Virginia -- Fiction', 'Repentance -- Fiction...",NaN,Moll Flanders,Defoe,http://www.wikidata.org/entity/Q612523,http://www.wikidata.org/entity/Q40946,1660.0,Daniel Defoe,Q612523,Q40946
188,PG9687,NaN,Robinson Crusoe,"Defoe, Daniel",NaN,1731.0,['en'],34,"{'Islands -- Fiction', 'Shipwreck survival -- ...",NaN,Robinson Crusoe,Defoe,http://www.wikidata.org/entity/Q483034,http://www.wikidata.org/entity/Q40946,1660.0,Daniel Defoe,Q483034,Q40946
189,PG9687,NaN,Robinson Crusoe,"Defoe, Daniel",NaN,1731.0,['en'],34,"{'Islands -- Fiction', 'Shipwreck survival -- ...",NaN,Robinson Crusoe,Defoe,http://www.wikidata.org/entity/Q483034,http://www.wikidata.org/entity/Q40946,1660.0,Daniel Defoe,Q483034,Q40946
